In [24]:
import numpy as np
from numpy.core.defchararray import replace

from keras.models import Sequential
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, Conv2D, Flatten, BatchNormalization, Conv2DTranspose

MINE = -1
COVERED = -2

SIZE_X = 16
SIZE_Y = 16
N_MINES = 40

class Board:
    def __init__(self, height, width, mines):
        self.visual_board = np.full((height, width), COVERED)
        self.height = height
        self.width = width
        self.mines_left = mines
        
        self.initialize_board()
        
    def initialize_board(self):
        self.board = np.zeros((self.height, self.width), dtype=np.int16)

        coords = np.array([[(i, j) for j in range(self.width)] for i in range(self.height)])
        coords = coords.reshape(coords.shape[0]*coords.shape[1], coords.shape[2])
                
        self.mines_location = coords[np.random.choice(len(coords), self.mines_left, replace=False)]
        set_mines = set(map(tuple, self.mines_location))
        self.non_mines_location = np.array(list(filter(lambda x : tuple(x) not in set_mines, coords)))
        
        for mine_coord in self.mines_location:
            self.board[mine_coord[0]][mine_coord[1]] = MINE
        
        for mine_coord in self.mines_location:            
            if mine_coord[0] > 0:
                if self.board[mine_coord[0]-1][mine_coord[1]] != MINE:
                    self.board[mine_coord[0]-1][mine_coord[1]] += 1
                
                if mine_coord[1] < self.width -1:
                    if self.board[mine_coord[0]-1][mine_coord[1]+1] != MINE:
                        self.board[mine_coord[0]-1][mine_coord[1]+1] += 1

                if mine_coord[1] > 0:
                    if self.board[mine_coord[0]-1][mine_coord[1]-1] != MINE:
                        self.board[mine_coord[0]-1][mine_coord[1]-1] += 1
            
            if mine_coord[0] < self.height -1:
                if self.board[mine_coord[0]+1][mine_coord[1]] != MINE:
                    self.board[mine_coord[0]+1][mine_coord[1]] += 1
                
                if mine_coord[1] < self.width -1:
                    if self.board[mine_coord[0]+1][mine_coord[1]+1] != MINE:
                        self.board[mine_coord[0]+1][mine_coord[1]+1] += 1

                if mine_coord[1] >= 0:
                    if self.board[mine_coord[0]+1][mine_coord[1]-1] != MINE:
                        self.board[mine_coord[0]+1][mine_coord[1]-1] += 1

            if mine_coord[1] < self.width - 1:
                if self.board[mine_coord[0]][mine_coord[1]+1] != MINE:
                    self.board[mine_coord[0]][mine_coord[1]+1] += 1

            if mine_coord[1] > 0:
                if self.board[mine_coord[0]][mine_coord[1]-1] != MINE:
                    self.board[mine_coord[0]][mine_coord[1]-1] += 1
                                    
    def print_board(self):
        for line in self.visual_board:
            for element in line:
                if element == COVERED:
                    print("| |",end=" ")
                else:
                    print("|"+str(element)+"|", end=" ")
            print("")
    
    def cover_board(self):
        self.visual_board = np.full((self.height, self.width), COVERED)
    
    def get_mine_mask(self):
        out_mtx = np.zeros((self.height, self.width))
        
        for mine in self.mines_location:
            out_mtx[mine[0]][mine[1]] = 1
        
        return out_mtx

    def generate_random_state(self, percentage_to_uncover):
        self.initialize_board()
        self.cover_board()
        
        mines_to_uncover = int((self.height * self.width - self.mines_left) * percentage_to_uncover)
        
        coords_uncovered = self.non_mines_location[np.random.choice(len(self.non_mines_location),\
                                                                    mines_to_uncover, replace=False)]
        for coord in coords_uncovered:
            self.visual_board[coord[0]][coord[1]] = self.board[coord[0]][coord[1]]
        
        return self.visual_board, self.get_mine_mask()

    def __uncover_recursive(self, x, y):
        if (self.height <= x) or (x < 0) or\
            (self.width <= y) or (y < 0) or\
            (self.board[x][y] == MINE or self.visual_board[x][y] != COVERED):
            return
        
        self.visual_board[x][y] = self.board[x][y]
        
        if self.board[x][y] == 0:
        
            self.__uncover_recursive(x+1, y)
            self.__uncover_recursive(x, y+1)
            self.__uncover_recursive(x+1, y+1)

            self.__uncover_recursive(x-1, y)
            self.__uncover_recursive(x, y-1)
            self.__uncover_recursive(x-1, y-1)

            self.__uncover_recursive(x+1, y-1)
            self.__uncover_recursive(x-1, y+1)
        
    
    def click_at(self, x, y):
        if self.board[x][y] == MINE:
            return True
        
        self.__uncover_recursive(x, y)
        
        return False

In [5]:
def generate_dataset(size, board, clicks_max = 10):
    X, Y = [], []
    
    for i in range(int(size/2)):
        clicks = 0
        clicks_limit = np.random.randint(1, clicks_max)
        while clicks != clicks_limit:
            x_rand = np.random.randint(0, board.height)
            y_rand = np.random.randint(0, board.width)
            
            if not board.click_at(x_rand, y_rand):
                clicks += 1
            else:
                board.cover_board()
        
        x, y = board.visual_board, board.get_mine_mask()
        x_mines = np.ones((board.height, board.width))*board.mines_left / (board.height* board.width)
        X.append(np.stack((x, x_mines), axis=2))
        Y.append(y)
        
        prob = np.random.random()
        
        x, y = board.generate_random_state(prob)
        
        X.append(np.stack((x, x_mines), axis=2))
        Y.append(y)
        
        print("{}/{}\r".format(i*2, size), end="")
            
    return np.array(X).reshape(size, board.height, board.width, 2), np.array(Y)

In [25]:
def instantiate_model(n, m):
    model = Sequential()

    model.add(Conv2D(32, kernel_size = 2 , activation='relu', input_shape=(n, m, 2), padding='same'))
    model.add(Conv2D(32, kernel_size = 2, activation='relu', padding='same'))
    model.add(Conv2D(64, kernel_size = 2, activation='relu', padding='same'))
    model.add(BatchNormalization())

    model.add(Conv2D(64, kernel_size = 3 , activation='relu', padding='same'))
    model.add(Conv2D(64, kernel_size = 3, activation='relu', padding='same'))
    model.add(Conv2D(64, kernel_size = 3, activation='relu', padding='same'))
    model.add(BatchNormalization())

    model.add(Conv2D(128, kernel_size = 5 , activation='relu', padding='same'))
    model.add(Conv2D(128, kernel_size = 5, activation='relu', padding='same'))
    model.add(Conv2D(128, kernel_size = 5, activation='relu', padding='same'))
    model.add(BatchNormalization())
    
    model.add(Conv2DTranspose(1, kernel_size = 5, activation='sigmoid', padding='same'))
    
    print(model.summary())

    model.compile(optimizer='rmsprop', loss='mse', metrics=['mse', 'mae'])

    return model

In [4]:
b = Board(SIZE_X, SIZE_Y, N_MINES)

In [37]:
b.click_at(2, 0)

False

In [38]:
b.print_board()

| | | | | | | | | | | | | | | | 
| | | | | | | | | | | | | | | | 
|2| | | | | | | | | | | | | | | 
| | | | | | | | | | | | | | | | 
| | | | | | | | | | | | | | | | 
| | | | | | | | | | | | | | | | 
| | | | | | | | | | | | | | | | 
| | | | | | | | | | | | | | | | 


In [39]:
b.board

array([[-1,  2,  1,  2, -1,  2,  0,  0],
       [ 3, -1,  2,  2, -1,  3,  1,  1],
       [ 2, -1,  3,  2,  3, -1,  1,  0],
       [ 2,  2,  3, -1,  2,  1,  1,  0],
       [ 1, -1,  2,  1,  1,  0,  0,  0],
       [ 2,  2,  1,  0,  0,  0,  0,  0],
       [-1,  2,  0,  0,  0,  0,  0,  0],
       [-1,  2,  0,  0,  0,  0,  0,  1]], dtype=int16)

In [40]:
b.visual_board

array([[-2, -2, -2, -2, -2, -2, -2, -2],
       [-2, -2, -2, -2, -2, -2, -2, -2],
       [ 2, -2, -2, -2, -2, -2, -2, -2],
       [-2, -2, -2, -2, -2, -2, -2, -2],
       [-2, -2, -2, -2, -2, -2, -2, -2],
       [-2, -2, -2, -2, -2, -2, -2, -2],
       [-2, -2, -2, -2, -2, -2, -2, -2],
       [-2, -2, -2, -2, -2, -2, -2, -2]])

In [7]:
X, Y = generate_dataset(100000, b)

In [10]:
import pickle

In [12]:
pickle.dump([X, Y] , open("mine_data_16_16_40", 'wb'))

In [26]:
model = instantiate_model(SIZE_X, SIZE_Y)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_45 (Conv2D)           (None, 16, 16, 32)        288       
_________________________________________________________________
conv2d_46 (Conv2D)           (None, 16, 16, 32)        4128      
_________________________________________________________________
conv2d_47 (Conv2D)           (None, 16, 16, 64)        8256      
_________________________________________________________________
batch_normalization_15 (Batc (None, 16, 16, 64)        256       
_________________________________________________________________
conv2d_48 (Conv2D)           (None, 16, 16, 64)        36928     
_________________________________________________________________
conv2d_49 (Conv2D)           (None, 16, 16, 64)        36928     
_________________________________________________________________
conv2d_50 (Conv2D)           (None, 16, 16, 64)       

In [28]:
model.fit(X, Y, epochs=100, batch_size=64)

Epoch 1/100
1563/1563 [==============================] - 27s 17ms/step - loss: 0.0487 - mse: 0.0487 - mae: 0.0954
Epoch 2/100
1563/1563 [==============================] - 27s 17ms/step - loss: 0.0484 - mse: 0.0484 - mae: 0.0948
Epoch 3/100
1563/1563 [==============================] - 27s 17ms/step - loss: 0.0481 - mse: 0.0481 - mae: 0.0942
Epoch 4/100
1563/1563 [==============================] - 27s 17ms/step - loss: 0.0479 - mse: 0.0479 - mae: 0.0937
Epoch 5/100
1563/1563 [==============================] - 27s 17ms/step - loss: 0.0476 - mse: 0.0476 - mae: 0.0932
Epoch 6/100
1563/1563 [==============================] - 27s 17ms/step - loss: 0.0474 - mse: 0.0474 - mae: 0.0927
Epoch 7/100
1563/1563 [==============================] - 27s 17ms/step - loss: 0.0472 - mse: 0.0472 - mae: 0.0922
Epoch 8/100
1563/1563 [==============================] - 27s 17ms/step - loss: 0.0470 - mse: 0.0470 - mae: 0.0918
Epoch 9/100
1563/1563 [==============================] - 27s 17ms/step - loss: 0.0468 - 

KeyboardInterrupt: 

In [29]:
model.save("mine_net.h5")

In [30]:
x, y = b.generate_random_state(0.5)
x_mines = np.ones((b.height, b.width))*b.mines_left / (b.height* b.width)
x = np.stack((x, x_mines), axis=2)

In [31]:
b.print_board()

| | | | | | |0| | | |1| | | | | |0| | | | | |1| | | | | | | | | 
|1| |1| | | |0| | | |1| | | |1| |0| | | |1| |2| | | |1| |0| | | 
|1| | | |1| | | | | |1| |1| |1| | | |1| | | | | |2| |1| |0| | | 
| | | | | | | | |0| |1| |1| |1| | | |1| | | | | | | |2| | | |1| 
|0| |1| | | | | |0| | | | | | | | | | | | | |2| | | | | | | |1| 
| | | | | | |2| | | | | |1| | | |0| | | |0| | | |1| | | | | |1| 
| | | | | | | | | | |2| |0| | | |0| |1| |1| | | |1| | | |1| | | 
| | |2| | | |3| | | |2| |0| |0| |0| | | | | | | | | |0| | | | | 
| | | | | | | | |2| | | | | | | |0| |2| | | | | | | | | |0| | | 
|3| |4| | | |1| | | | | |1| |0| |1| | | |3| |2| | | | | |1| |1| 
| | | | |2| | | |1| | | |1| |0| | | | | | | |1| | | |1| | | | | 
|2| | | | | |0| |0| |1| |1| | | | | | | | | | | |1| | | |2| |3| 
|2| |3| | | |1| | | | | | | | | | | |1| | | | | | | | | |2| | | 
| | | | | | |1| | | | | | | | | |1| | | |0| | | |2| | | |3| |2| 
| | |5| |3| |2| | | | | | | |3| | | | | |1| | | | | |3| | | |3| 
| | | | | | | | |0| |1| |

In [32]:
for i, line in enumerate(model.predict(x.reshape(1, 16, 16, 2))[0]):
    print("({}, 0) => ".format(i), end='')
    for j, ele in enumerate(line):
        print("|{0:.2f}|".format(ele[0]), end="")
    print("")

(0, 0) => |0.00||0.00||0.00||0.00||0.00||0.00||0.00||0.00||0.00||0.00||0.00||0.00||1.00||0.00||0.00||0.00|
(1, 0) => |0.00||0.00||0.00||0.00||0.00||0.00||1.00||0.00||0.00||0.00||0.00||0.00||0.00||0.00||0.00||0.00|
(2, 0) => |0.00||1.00||0.00||0.00||0.00||0.00||0.00||0.00||0.00||0.00||0.00||1.00||0.00||0.00||0.00||0.00|
(3, 0) => |0.00||0.00||1.00||0.00||0.00||0.00||0.00||0.00||0.00||0.00||1.00||0.03||1.00||0.00||0.00||0.00|
(4, 0) => |0.00||0.00||0.00||0.00||0.00||0.00||1.00||0.00||0.00||0.00||0.00||0.00||0.00||0.00||1.00||0.00|
(5, 0) => |0.00||0.00||0.00||0.00||0.00||0.00||0.00||0.00||0.00||0.00||0.00||0.00||0.00||1.00||0.00||0.00|
(6, 0) => |0.09||0.00||0.00||0.00||1.00||0.00||0.00||0.00||0.00||0.00||0.00||0.00||0.00||0.00||0.00||0.00|
(7, 0) => |0.00||0.00||0.00||0.00||1.00||0.00||0.00||0.00||0.00||0.00||1.00||0.00||0.00||0.00||0.00||0.00|
(8, 0) => |1.00||1.00||0.00||0.00||0.00||0.00||0.00||0.00||0.00||0.00||1.00||0.00||0.00||0.00||0.00||0.00|
(9, 0) => |0.00||0.00||1.00||0.00||0.

In [33]:
for line in y:
    for ele in line:
        print("|{0:.2f}|".format(ele), end=" ")
    print("")

|0.00| |0.00| |0.00| |0.00| |0.00| |0.00| |0.00| |0.00| |0.00| |0.00| |0.00| |0.00| |1.00| |0.00| |0.00| |0.00| 
|0.00| |0.00| |0.00| |0.00| |0.00| |0.00| |1.00| |0.00| |0.00| |0.00| |0.00| |0.00| |0.00| |0.00| |0.00| |0.00| 
|0.00| |1.00| |0.00| |0.00| |0.00| |0.00| |0.00| |0.00| |0.00| |0.00| |1.00| |0.00| |0.00| |0.00| |0.00| |0.00| 
|0.00| |0.00| |0.00| |0.00| |0.00| |0.00| |0.00| |0.00| |0.00| |0.00| |0.00| |1.00| |1.00| |0.00| |0.00| |0.00| 
|0.00| |0.00| |1.00| |0.00| |0.00| |0.00| |1.00| |0.00| |0.00| |0.00| |0.00| |0.00| |0.00| |0.00| |1.00| |0.00| 
|0.00| |0.00| |0.00| |0.00| |0.00| |0.00| |0.00| |0.00| |0.00| |0.00| |0.00| |0.00| |0.00| |1.00| |0.00| |0.00| 
|0.00| |0.00| |0.00| |0.00| |1.00| |0.00| |0.00| |0.00| |0.00| |0.00| |0.00| |0.00| |0.00| |0.00| |0.00| |0.00| 
|0.00| |0.00| |1.00| |0.00| |1.00| |0.00| |0.00| |0.00| |0.00| |0.00| |1.00| |0.00| |0.00| |0.00| |0.00| |0.00| 
|0.00| |1.00| |0.00| |0.00| |0.00| |0.00| |0.00| |0.00| |0.00| |0.00| |1.00| |0.00| |0.00| |0.00

In [494]:
x = np.full((8, 8), -2)

In [495]:
x[1][3] = 2
x[1][4] = 1
x[1][5] = 1
x[1][6] = 1
x[1][7] = 1

x[2][3] = 2
x[2][4] = 0
x[2][5] = 0
x[2][6] = 0
x[2][7] = 0


x[3][3] = 1
x[3][4] = 0
x[3][5] = 0
x[3][6] = 0
x[3][7] = 0

In [496]:
x

array([[-2, -2, -2, -2, -2, -2, -2, -2],
       [-2, -2, -2,  2,  1,  1,  1,  1],
       [-2, -2, -2,  2,  0,  0,  0,  0],
       [-2, -2, -2,  1,  0,  0,  0,  0],
       [-2, -2, -2, -2, -2, -2, -2, -2],
       [-2, -2, -2, -2, -2, -2, -2, -2],
       [-2, -2, -2, -2, -2, -2, -2, -2],
       [-2, -2, -2, -2, -2, -2, -2, -2]])